<a href="https://colab.research.google.com/github/alkarps/GB.Methods-for-collecting-and-processing-data-from-the-Internet/blob/hw03/hw03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install bs4
#!pip install requests

In [2]:
from bs4 import BeautifulSoup as bs
from collections import namedtuple
from pprint import pprint
from time import sleep
import requests
import json
import re

In [3]:
def saveJsonTo(file_name, income):
  json_string = json.dumps(income)
  with open(file_name, 'w') as outfile:
      outfile.write(json_string)

def readAndPrintJsonFrom(file_name):
  with open(file_name) as json_file:
    data = json.load(json_file)
    print(data)

In [4]:
Vacancy = namedtuple("Vacancy", "source, name, link, min_salary, max_salary")
Salary = namedtuple("Salary", "amount, currency")

def create_salary(amount, currency):
  if isinstance(amount, str):
    return Salary(int(amount.replace("\u202f","")), currency.strip())
  else:
    return Salary(int(amount), currency.strip())

In [5]:
def get_soup_by_url(url, payload):
  headers = {
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
      }
  res = requests.get(url, headers=headers, params=payload)
  if res.ok:
    return bs(res.content, 'html.parser')
  else:
    return None

**Собрать информацию о вакансиях на вводимую должность с сайтов hh.ru, SuperJob и/или работа.ру. Приложение должно анализировать несколько страниц сайта. Полученный список должен содержать в себе минимум:**



*   Наименование вакансии.
*   Предполагаемую зарплату (**дополнительно**: разносим на три поля: минимульную, максимальную и валюту. Цифры приводим к числовому формату). 
*   Ссылку на саму вакансию.
*   Сайт, откуда собрана вакансия.


По желанию, можно добавить еще параметров (например, работодатели в предложении). Структура должна быть одинакова для ванаксий со всех сайтов. Общий результат можно вывести в помощью dataFrame, сохранить в json или csv формат.

In [6]:
def get_hh_ru_soup(vacancy_name, page):
  return get_soup_by_url("https://hh.ru/search/vacancy", {"text": vacancy_name, 'page': page, 'hhtmFrom': 'vacancy_search_list'})

In [7]:
def parse_hh_page(soup):
  result = list()
  source = "hh.ru"
  salary_reg = r'\d+[ ]?\d+'
  vacancy_cards = soup.find_all("div", {"class": "vacancy-serp-item-body__main-info"})
  for vacancy_card in vacancy_cards:
    vacancy = vacancy_card.find("a", {"class": "serp-item__title"})
    link = vacancy["href"]
    title = vacancy.text
    salary = vacancy_card.find("span",{"class": "bloko-header-section-3"})
    min_salary = None
    max_salary = None
    if salary:
      salary = salary.text
      amounts = re.findall(salary_reg, salary)
      currency = re.split(salary_reg, salary)[-1]
      if salary.startswith("от"):
        min_salary = create_salary(amounts[0], currency)
      elif salary.startswith("до"):
        max_salary = create_salary(amounts[0], currency)
      else:
        min_salary = create_salary(amounts[0], currency)
        max_salary = create_salary(amounts[1], currency)
    result.append(Vacancy(source, title, link, min_salary, max_salary))
  return result

In [8]:
def parse_hh(vacancy_name):
  result = list()
  for i in range(40):
    soup = get_hh_ru_soup(vacancy_name, str(i))
    if soup:
      vacancies = parse_hh_page(soup)
      if(len(vacancies)==0):
        break
      sleep(1)
      result.extend(vacancies)
    else:
      break
  return result

In [9]:
vacancies = parse_hh("тестер")

In [10]:
saveJsonTo("parsed_vacancies.json", vacancies)

In [11]:
readAndPrintJsonFrom("parsed_vacancies.json")

[['hh.ru', 'Тестировщик CRM', 'https://hh.ru/vacancy/74077174?from=vacancy_search_list&query=%D1%82%D0%B5%D1%81%D1%82%D0%B5%D1%80', [140000, 'руб.'], None], ['hh.ru', 'QA Engineer Auto (Java)', 'https://hh.ru/vacancy/72076374?from=vacancy_search_list&query=%D1%82%D0%B5%D1%81%D1%82%D0%B5%D1%80', None, None], ['hh.ru', 'QA Automation engineer (зарплатный проект)', 'https://hh.ru/vacancy/73180600?from=vacancy_search_list&query=%D1%82%D0%B5%D1%81%D1%82%D0%B5%D1%80', None, None], ['hh.ru', 'Middle / Senior QA инженер', 'https://hh.ru/vacancy/47759634?from=vacancy_search_list&query=%D1%82%D0%B5%D1%81%D1%82%D0%B5%D1%80', [200000, 'руб.'], None], ['hh.ru', 'Senior QA инженер', 'https://hh.ru/vacancy/49887152?from=vacancy_search_list&query=%D1%82%D0%B5%D1%81%D1%82%D0%B5%D1%80', [200000, 'руб.'], None], ['hh.ru', 'Middle / Senior QA в команду hr-брендинговые продукты', 'https://hh.ru/vacancy/69811697?from=vacancy_search_list&query=%D1%82%D0%B5%D1%81%D1%82%D0%B5%D1%80', [200000, 'руб.'], None], [